In [1]:
# Import Dependencies
import time as time
import pandas as pd
import requests
import pymongo
import re
from collections import defaultdict
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup


In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.mars_db

In [4]:
# Define function for initializing the chrome browser

def init_browser():
    executable_path = {'executable_path': 'chromedriver.exe'}
    return Browser('chrome', **executable_path, headless=False)

# Set path to chrome driver, and set browser type
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
# Set URLs
url_news = 'https://mars.nasa.gov/news/'
url_img = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
url_wx = 'https://twitter.com/marswxreport?lang=en'
url_facts = 'http://space-facts.com/mars/'
url_hemi = 'http://web.archive.org/web/20181114171728/https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [6]:
# Create dictionary to store output
#mars_dict = url_output

In [7]:
# Set variables
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [8]:
# Create queries for each URL

# Query for Mars news 
browser.visit(url_news)
response_news = requests.get(url_news)
soup = BeautifulSoup(response_news.text, 'html.parser')

news_title = soup.find('div', class_="content_title").text
news_p = soup.find('div', class_="rollover_description_inner").text
print(news_title)
print(news_p)



NASA's InSight Places First Instrument on Mars



In deploying its first instrument onto the surface of Mars, the lander completes a major mission milestone.



In [9]:
# Query for Mars Image
browser.visit(url_img)
response_img = requests.get(url_img)
soup = BeautifulSoup(response_img.text, 'html.parser')

# Use splinter to navigate to the full image details page
xpath = '//*[@id="full_image"]'
browser.find_by_xpath(xpath).first.click()
time.sleep(2)
xpath2 = '//*[@id="fancybox-lock"]/div/div[2]/div/div[1]/a[2]'
browser.find_by_xpath(xpath2).first.click()
time.sleep(2)
xpath3 = '//*[@id="page"]/section[1]/div/article/figure/a/img'
browser.find_by_xpath(xpath3).first.click()
featured_image_url = browser.url
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA20464_hires.jpg


In [10]:
# Query for Mars Weather
browser.visit(url_wx)
response_wx = requests.get(url_wx)
soup = BeautifulSoup(response_wx.text, 'html.parser')

mars_weather_gather = soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text

if "Sol" and "pressure" in soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text:
    mars_weather = mars_weather_gather
else: 
    mars_weather = soup.find_next_sibling('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text

print(mars_weather)

Sol 2271 (2018-12-26), high -7C/19F, low -70C/-93F, pressure at 8.31 hPa, daylight 06:41-18:54


In [11]:
# Query for Mars Facts
browser.visit(url_facts)
response_facts = requests.get(url_facts)
soup = BeautifulSoup(response_facts.text, 'html.parser')

mars_facts = soup.find('table', id='tablepress-mars').text
mars_facts

'\n\n\nEquatorial Diameter:6,792 km\n\n\n\nPolar Diameter:6,752 km\n\n\n\nMass:6.42 x 10^23 kg (10.7% Earth)\n\n\nMoons:2 (Phobos & Deimos)\n\n\nOrbit Distance:227,943,824 km (1.52 AU)\n\n\nOrbit Period:687 days (1.9 years)\n\n\n\nSurface Temperature: -153 to 20 °C\n\n\nFirst Record:2nd millennium BC\n\n\nRecorded By:Egyptian astronomers\n\n\n'

In [12]:
# Convert mars_facts into list
mars_facts_list = mars_facts.split('\n\n\n')
mars_facts_list = list(map(lambda mars_facts_list: mars_facts_list.strip(), mars_facts_list))
#mars_facts_list_split = [mars_facts_list.split(':', 1)[0] for i in mars_facts_list]
mars_facts_list = [i.split(':') for i in mars_facts_list] 
mars_facts_list.pop(0)
mars_facts_list.pop(9)


['']

In [13]:
mars_facts_list
mars_facts_list_flat = [val for sublist in mars_facts_list for val in sublist]
mars_facts_list

[['Equatorial Diameter', '6,792 km'],
 ['Polar Diameter', '6,752 km'],
 ['Mass', '6.42 x 10^23 kg (10.7% Earth)'],
 ['Moons', '2 (Phobos & Deimos)'],
 ['Orbit Distance', '227,943,824 km (1.52 AU)'],
 ['Orbit Period', '687 days (1.9 years)'],
 ['Surface Temperature', ' -153 to 20 °C'],
 ['First Record', '2nd millennium BC'],
 ['Recorded By', 'Egyptian astronomers']]

In [14]:
# Load table data into dataframe
mars_facts_df = pd.DataFrame(data = mars_facts_list)
mars_facts_df.columns = ["Criteria", "Facts"]
mars_facts_df

,Criteria,Facts
0,Equatorial Diameter,"6,792 km"
1,Polar Diameter,"6,752 km"
2,Mass,6.42 x 10^23 kg (10.7% Earth)
3,Moons,2 (Phobos & Deimos)
4,Orbit Distance,"227,943,824 km (1.52 AU)"
5,Orbit Period,687 days (1.9 years)
6,Surface Temperature,-153 to 20 °C
7,First Record,2nd millennium BC
8,Recorded By,Egyptian astronomers


In [15]:
# Convert df to HTML table string
#mars_facts_html = mars_facts_df.to_html('mars_facts.html')
#mars_facts_html

In [16]:
# Convert df to HTML table string
mars_facts_html = mars_facts_df.to_html()
mars_facts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Criteria</th>\n      <th>Facts</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record</td>\n      <td>2nd millennium BC</td>\n 

In [35]:
img_url = []
title = []
url_hemi = 'http://web.archive.org/web/20181114171728/https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(url_hemi)
response_hemi = requests.get(url_hemi)

# Create soup object from html
soup = BeautifulSoup(response_hemi.text, 'html.parser')
result = soup.find_all('h3')

# Prepare string
title_str = str(result)
title_str = title_str.replace("<h3>", "")
title_str = title_str.replace("</h3>", "")
title_str = title_str.replace("[", "")
title_str = title_str.replace("]", "")
title_str = title_str.strip()
if "Hemisphere" not in title_str:  
    title_str = title_str.replace("Enhanced", "Hemisphere")
else:
    pass
title = title_str.split(",")

xpath = '//*[@id="product-section"]/div[2]/div[1]/div/a'
browser.find_by_xpath(xpath).first.click()
time.sleep(1)
xpath2 = '//*[@id="wide-image"]/div/ul/li[1]/a'
browser.find_by_xpath(xpath2).first.click()
time.sleep(1)
#browser.back()
url_1 = browser.url
browser.visit(url_1)
response_1 = requests.get(url_1)
soup1 = BeautifulSoup(response_1.text, 'html.parser')
soup1.find("img", class_="wide-image")["src"]
web + img_src = final_src
img_url.append(final_src)

SyntaxError: can't assign to operator (<ipython-input-35-efa4831930ee>, line 39)

In [39]:
img_src = soup1.find("img", class_="wide-image")["src"]
final_src = "http://web.archive.org" + img_src 
final_src

'http://web.archive.org/web/20181114182238im_/https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'

In [ ]:
soup.find(src).text)

In [ ]:
### Scrape Mars Hemisphere images ###
img_url = []
title = []
url_hemi = 'http://web.archive.org/web/20181114171728/https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(url_hemi)
response_hemi = requests.get(url_hemi)

# Create soup object from html
soup = BeautifulSoup(response_hemi.text, 'html.parser')
result = soup.find_all('h3')

# Prepare string
title_str = str(result)
title_str = title_str.replace("<h3>", "")
title_str = title_str.replace("</h3>", "")
title_str = title_str.replace("[", "")
title_str = title_str.replace("]", "")
title_str = title_str.strip()
if "Hemisphere" not in title_str:  
    title_str = title_str.replace("Enhanced", "Hemisphere")
else:
    pass
title = title_str.split(",")

# Query for Cerberus Hemisphere
## The daggum wayback website changed how it was feeding me pages 
## (by adding a pop-up banner), late at night. I had to redo this 
## section; i'm leaving in the old code in case it goes back to the
## old way.

xpath = '//*[@id="product-section"]/div[2]/div[1]/div/a'
browser.find_by_xpath(xpath).first.click()
time.sleep(1)
xpath2 = '//*[@id="wide-image"]/div/ul/li[1]/a'
browser.find_by_xpath(xpath2).first.click()
time.sleep(1)
#browser.back()
url_1 = browser.url
browser.visit(url_1)
response_1 = requests.get(url_1)
soup1 = BeautifulSoup(response_1.text, 'html.parser')
img_src = soup1.find("img", class_="wide-image")["src"]
final_src = "http://web.archive.org" + img_src
img_url.append(final_src) 
browser.back()

# xpath3 = '//*[@id="wm-expand"]/span[2]'
# browser.find_by_xpath(xpath3).first.click()
# time.sleep(1)
# xpath4 = '//*[@id="wm-capresources"]/div/a'
# browser.find_by_xpath(xpath4).first.click()
# browser.back()
# browser.back()

# Query for Schiaparelli Hemisphere
xpath = '//*[@id="product-section"]/div[2]/div[2]/div/a'
browser.find_by_xpath(xpath).first.click()
time.sleep(1)
xpath2 = '//*[@id="wide-image"]/div/ul/li[1]/a'
browser.find_by_xpath(xpath2).first.click()
time.sleep(1)
#browser.back()
url_1 = browser.url
browser.visit(url_1)
response_1 = requests.get(url_1)
soup1 = BeautifulSoup(response_1.text, 'html.parser')
img_src = soup1.find("img", class_="wide-image")["src"]
final_src = "http://web.archive.org" + img_src
img_url.append(final_src) 
browser.back()

# xpath3 = '//*[@id="wm-expand"]/span[2]'
# browser.find_by_xpath(xpath3).first.click()
# time.sleep(1)
# xpath4 = '//*[@id="wm-capresources"]/div/a'
# browser.find_by_xpath(xpath4).first.click()
# browser.back()
# browser.back()

# Query for Syrtis Major Hemisphere
xpath = '//*[@id="product-section"]/div[2]/div[3]/div/a'
browser.find_by_xpath(xpath).first.click()
time.sleep(1)
xpath2 = '//*[@id="wide-image"]/div/ul/li[1]/a'
browser.find_by_xpath(xpath2).first.click()
time.sleep(1)
#browser.back()
url_1 = browser.url
browser.visit(url_1)
response_1 = requests.get(url_1)
soup1 = BeautifulSoup(response_1.text, 'html.parser')
img_src = soup1.find("img", class_="wide-image")["src"]
final_src = "http://web.archive.org" + img_src
img_url.append(final_src) 
browser.back()

# xpath3 = '//*[@id="wm-expand"]/span[2]'
# browser.find_by_xpath(xpath3).first.click()
# time.sleep(1)
# xpath4 = '//*[@id="wm-capresources"]/div/a'
# browser.find_by_xpath(xpath4).first.click()
# browser.back()
# browser.back()

# Query for Valles Marineris Hemisphere
xpath = '//*[@id="product-section"]/div[2]/div[4]/div/a'
browser.find_by_xpath(xpath).first.click()
time.sleep(1)
xpath2 = '//*[@id="wide-image"]/div/ul/li[1]/a'
browser.find_by_xpath(xpath2).first.click()
time.sleep(1)

# xpath3 = '//*[@id="wm-expand"]/span[2]'
# browser.find_by_xpath(xpath3).first_click()
# time.sleep(1)
# xpath4 = '//*[@id="wm-capresources"]/div/a'
# time.sleep(1)

url_1 = browser.url
browser.visit(url_1)
response_1 = requests.get(url_1)
soup1 = BeautifulSoup(response_1.text, 'html.parser')
img_src = soup1.find("img", class_="wide-image")["src"]
final_src = "http://web.archive.org" + img_src
img_url.append(final_src) 
# browser.windows.current = browser.windows[4]
# img_url.append(browser.url)
# browser.windows.current = browser.windows[3]
# img_url.append(browser.url)
# browser.windows.current = browser.windows[2]
# img_url.append(browser.url)
# browser.windows.current = browser.windows[1]
# img_url.append(browser.url)

In [ ]:
img_url

In [ ]:
mars_hemi_df = pd.DataFrame(
    {'title': title,
     'img_url': img_url,
    })
mars_hemi_df.set_index("title")

In [ ]:
title_df

In [ ]:
column_names = ['title']
column_values = 

for i in ('apple', 'banana', 'carrot'):
    fruitdict[i] = locals()[i]title
name_value_tuples = zip(column_names, column_values)
title_dict = {}
for key, value in name_value_tuples:
    if key in title_dict:
        pass # Insert logic for handling duplicate keys
    else:
        title_dict[key] = value
        

        
column_names = ['img_url', 'img_url', 'img_url', 'img_url']
column_values = img_url
name_value_tuples = zip(column_names, column_values)
img_url_dict = {}
for key, value in name_value_tuples:
    if key in img_url_dict:
        pass # Insert logic for handling duplicate keys
    else:
        img_url_dict[key] = value

In [ ]:
#mars_hemi_images_dict = dict(zip(title, img_url))
#mars_hemi_images_dict

In [ ]:
title_str = str(title)
img_url_str = str(img_url)

In [ ]:
mars_dict_5 = {}
mars_dict_5["title"] = title_str
mars_dict_5["img_urg"] = img_url_str
#mars_dict_list_5 = []
#mars_dict_list_5.append(mars_dict_5.copy())
#mars_dict_list_5
mars_dict_6 = {k: v for k, v in zip(title, img_url)}
mars_dict_6
#for n in mars_dict_list_5:
#    mars_dict_list_5 = collections.defaultdict(dict)
#    mars_dict_list_5["title"]["img_url"] = str(n)
#    ultimate_list.append(node_dict)

In [ ]:
#title = ["title: " + s for s in title]
title = [" " + s for s in title]
img_url = ["img_url: " + s for s in img_url]
title

In [ ]:
hemi_dict = {k: v for k, v in zip(title, img_url)}
#hemi_dict_list = list(hemi_dict)
hemi_dict

In [ ]:
hemi_dict

In [ ]:
def split_dict_equally(input_dict, chunks=4):
    "Splits dict by keys. Returns a list of dictionaries."
    # prep with empty dicts
    return_list = [dict() for idx in range(chunks)]
    idx = 0
    for k,v in input_dict.items():
        return_list[idx][k] = v
        if idx < chunks-1:  # indexes start at 0
            idx += 1
        else:
            idx = 0
    return return_list

In [ ]:
split_dict_equally(hemi_dict, chunks=4)

In [ ]:
#hemi_dict
#re.sub('(:[^:]*),', r'\1,', hemi_dict)

hemi_list1 = [{hemi_dict} for k in range(3)]

In [ ]:
# this is kind of cool: will test if an alternative to list of dicts
hemi_list = hemi_dict.items()
hemi_list

for line in data_list

details = (img_url)

data_dict = defaultdict(list)
data_dict["title"].append()

empty_dict = dict.fromkeys(["title", "img_url"])

#for x in title:
#    empty_dict["title": x]

#dict.setdefault("title", []).append(title)
#for x in empty_dict.title():
#    empty_dict
empty_dict.update({"title" : img_url[0]})
empty_dict.update({"title" : img_url[1]})
empty_dict.update({"title" : img_url[2]})
empty_dict.update({"title" : img_url[3]})
img_url[1]

key_value = {}
my_dictionary = {}
list_key = ['title', 'title', 'title', 'title']
list_val = title
lenList = len(list_key)
my_dict_output = {}

for elements in range(0,lenList):
    key = list_key[elements]
    value = list_val[elements]
    #Storing the extracted key value pair in the dictionary
    key_value[key] = value
print(key_value)

# Convert 2 lists to dictionary
#empty_dict = dict.fromkeys(["title", "img_url"])
#dict.setdefault("title", []).append(title)

keys = ["title", "title", "title", "title"]
values = title.values()
#def create_dict(keys, values):
#    return dict(zip(keys, values + [None] * (len(keys) - len(values))))

title_dict = {k: v for k, v in zip(title, keys)}
title_dict2 = {k: v for v, k in title_dict.items()}
#new_dict = dict((v,k) for k,v in new_dict.items())

keys2 = ["img_url", "img_url", "img_url", "img_url"]
img_url
img_url_dict = {k: v for k, v in zip(img_url, keys2)}
img_url_dict1 = {k: v for v, k in zip(img_url, keys2)}
#{k: v for e in img_url for k, v in zip(('', 'post_author'), e)}

#img_url_dict = {k:v for (k,v) in img_url_dict}
title_dict2
img_url_dict1

In [ ]:
mars_dict = {}
mars_dict["Latest Mars News"] = news_title
mars_dict["News Paragraph"] = news_p
mars_dict["Featured Mars Image"] = featured_image_url
mars_dict["Current Weather on Mars"] = mars_weather
mars_dict["Mars Facts"] = mars_facts_df
mars_dict["Mars Hemispheres"] = hemi_list

#keys = ["news", "img", "wx", "facts", "hemi"]
#values = [news_title, news_p, featured_image_url, mars_weather, mars_facts_df, hemi_list]

##def scrape(values):
##    scrape_dict = dict.fromkeys(["news", "img", "wx", "facts", "hemi"])
##    #scrape_dict = dict(zip(news, img, wx, facts, hemi))
##    scrape_dict = dict(zip(values))
    
#scrape_dict = 

#empty_dict = dict.fromkeys(["news", "img", "wx", "facts", "hemi"])
#empty_dict
mars_dict
#scrape(values)

In [ ]:
mars_facts_html